In [19]:
import sqlite3
import pandas as pd
import requests
from IPython.display import display, Markdown

# --- CONFIGURACIÓN ---
DB_PATH = 'data/database/parlamento.db'
BOLETIN_ID = '2439-20'
pd.set_option('display.max_rows', 100) # Para mostrar más filas en las tablas largas

# --- 1. CONEXIÓN A LA BASE DE DATOS ---
try:
    conn = sqlite3.connect(DB_PATH)
    print(f"✅ Conexión a la base de datos exitosa.")
except sqlite3.Error as e:
    print(f"❌ Error de base de datos: {e}")
    # Si la conexión falla, no continuamos.
    conn = None

if conn:
    # --- 2. MOSTRAR TÍTULO ---
    display(Markdown(f"# 📖 Análisis del Proyecto de Ley: {BOLETIN_ID}"))

    # --- 3. OBTENER Y MOSTRAR FICHA PRINCIPAL ---
    display(Markdown("--- \n## ℹ️ Ficha Principal del Proyecto"))
    df_bill = pd.read_sql_query("SELECT * FROM bills WHERE bill_id = ?", conn, params=(BOLETIN_ID,))
    
    if df_bill.empty:
        display(Markdown(f"**Error:** No se encontró ningún proyecto con el boletín `{BOLETIN_ID}`."))
    else:
        # Transponer el DataFrame para una mejor visualización de la ficha
        display(df_bill.set_index('bill_id').T)

    # --- 4. OBTENER Y MOSTRAR AUTORES Y MATERIAS ---
    display(Markdown("--- \n## ✍️ Autores y Materias"))
    
    # Autores
    authors_query = """
        SELECT p.nombre_completo, p.genero
        FROM bill_authors ba
        JOIN dim_parlamentario p ON ba.mp_uid = p.mp_uid
        WHERE ba.bill_id = ?;
    """
    df_authors = pd.read_sql_query(authors_query, conn, params=(BOLETIN_ID,))
    display(Markdown("### Autores Parlamentarios"))
    if df_authors.empty:
        display(Markdown("_Iniciativa del Ejecutivo o no se encontraron autores parlamentarios en la BD._"))
    else:
        display(df_authors)
        
    # Materias
    materias_query = """
        SELECT m.nombre
        FROM bill_materias bm
        JOIN dim_materias m ON bm.materia_id = m.materia_id
        WHERE bm.bill_id = ?
    """
    df_materias = pd.read_sql_query(materias_query, conn, params=(BOLETIN_ID,))
    display(Markdown("### Materias Asociadas"))
    if df_materias.empty:
        display(Markdown("_No se encontraron materias específicas._"))
    else:
        display(df_materias)

    # --- 5. MOSTRAR TABLA DE TRÁMITES ---
    display(Markdown("--- \n## 🛤️ Historial de Tramitación (`bill_tramites`)"))
    tramites_query = """
        SELECT fecha_tramite, descripcion, etapa_especifica, camara, sesion
        FROM bill_tramites
        WHERE bill_id = ?
        ORDER BY fecha_tramite ASC
    """
    df_tramites = pd.read_sql_query(tramites_query, conn, params=(BOLETIN_ID,))
    if df_tramites.empty:
        display(Markdown("_No se encontró historial de trámites para este proyecto._"))
    else:
        display(df_tramites)

    # --- 6. MOSTRAR TABLA DE DOCUMENTOS Y UN EXTRACTO ---
    display(Markdown("--- \n## 📄 Documentos Asociados (`bill_documentos`)"))
    documentos_query = "SELECT documento_id, tipo_documento, url_documento, fecha_documento FROM bill_documentos WHERE bill_id = ? ORDER BY fecha_documento ASC"
    df_documentos = pd.read_sql_query(documentos_query, conn, params=(BOLETIN_ID,))
    
    if df_documentos.empty:
        display(Markdown("_No se encontraron documentos asociados para este proyecto en la base de datos._"))
    else:
        display(Markdown("### Tabla de Documentos"))
        display(df_documentos)
        
        # Tomar el primer documento de la lista como ejemplo
        doc_url = df_documentos['url_documento'].iloc[0]
        doc_id = df_documentos['documento_id'].iloc[0]
        
        display(Markdown(f"--- \n### Extracto del Documento ID: {doc_id} (primeros 1000 caracteres)"))
        display(Markdown(f"**URL:** `{doc_url}`"))
        
        try:
            print("\nDescargando contenido del texto de ejemplo...")
            response = requests.get(doc_url, timeout=30)
            response.raise_for_status()
            response.encoding = 'utf-8'
            texto_completo = response.text
            
            # Usamos un bloque <pre> para que el texto respete los saltos de línea y espacios
            display(Markdown(f"<pre style='background-color:#f4f4f4; padding: 10px; border-radius: 5px; white-space: pre-wrap;'>{texto_completo[:10000]}...</pre>"))

        except requests.RequestException as e:
            display(Markdown(f"**Error al descargar el documento de ejemplo:** `{e}`"))
            
    # Cerrar la conexión
    conn.close()
    print("\n✅ Conexión a la base de datos cerrada.")

✅ Conexión a la base de datos exitosa.


# 📖 Análisis del Proyecto de Ley: 2439-20

--- 
## ℹ️ Ficha Principal del Proyecto

bill_id,2439-20
titulo,"Sustituye ley N° 19.366, que sanciona el tráfi..."
resumen,None
tipo_proyecto,None
fecha_ingreso,1999-12-14
etapa,Tramitación terminada
subetapa,/
iniciativa,Mensaje
origen,C.Diputados
urgencia,Sin urgencia
resultado_final,Publicado


--- 
## ✍️ Autores y Materias

...

--- 
### Extracto del Documento ID: 23 (primeros 1000 caracteres)

**URL:** `http://datos.bcn.cl/recurso/cl/documento/668657.txt`


Descargando contenido del texto de ejemplo...


<pre style='background-color:#f4f4f4; padding: 10px; border-radius: 5px; white-space: pre-wrap;'>PROYECTO DE LEY:
"TÍTULO I
DE LOS DELITOS Y SANCIONES
Párrafo 1º
De los delitos generales
ARTICULO 1°.- Los que elaboren, fabriquen, transformen, preparen o extraigan sustancias o drogas estupefacientes o psicotrópicas productoras de dependencia física o síquica, capaces de provocar graves efectos tóxicos o daños considerables a la salud, sin la debida autorización, serán castigados con presidio mayor en su grado mínimo a medio y multa de cuarenta a cuatrocientas unidades tributarias mensuales. Dicha pena corporal podrá rebajarse en un grado de concurrir alguna de las circunstancias que menciona el artículo cuarto de esta ley; y si es, además, de tal entidad, que amerite dicha reducción.
Si se tratare de otras drogas o sustancias de esta índole que no produzcan los efectos indicados en el inciso anterior, podrá rebajarse la pena hasta en dos grados, de concurrir dichas circunstancias en los términos señalados precedentemente.
Se presumirá autor del delito sancionado en este artículo, a quien tenga en su poder elementos, instrumentos, materiales o equipos comúnmente utilizados en la elaboración, fabricación, preparación, transformación o extracción de las sustancias o drogas a que se refieren los incisos anteriores, si de los antecedentes reunidos, puede desprenderse que estuvieron destinados a tal finalidad.
ARTICULO 2°.- La producción, fabricación, elaboración, distribución, transporte, comercialización, importación, exportación, posesión o tenencia de sustancias químicas, a sabiendas de que se destinarán a la preparación de drogas estupefacientes o sustancias psicotrópicas para perpetrar, dentro o fuera del país, alguno de los hechos considerados como delitos en esta ley, será castigado con presidio menor en su grado máximo a presidido mayor en su grado mínimo y multa de cuarenta a cuatrocientas unidades tributarias mensuales.
ARTICULO 3°.- Las penas establecidas en el artículo primero, con las modalidades en él señaladas, se aplicarán también a los que trafiquen, a cualquier título, con las sustancias a que dicha disposición se refiere; o con las materias primas que sirvan para obtenerlas; y a los que, por cualquier medio, induzcan, promuevan o faciliten el uso o consumo de tales sustancias.
Se presumirá autor de dicho tráfico, a quien, careciendo de la debida autorización, importe, exporte, transporte, adquiera, transfiera, substraiga, posea, suministre, guarde o porte consigo tales sustancias o materias primas, a menos que justifique que están destinadas a un tratamiento médico o a su uso personal exclusivo y próximo en el tiempo.
En este último caso, le serán aplicables las normas de los artículos 48 y siguientes de esta ley.
En el caso de que el sentenciado no pagare la multa impuesta sufrirá, por vía de sustitución, la pena de reclusión, regulándose un día por cada media unidad tributaria mensual. En todo caso, la reclusión no podrá exceder de seis meses.
Sin perjuicio de lo dispuesto en el inciso anterior, en casos debidamente calificados, el tribunal podrá eximir al condenado del pago de multa o imponerle una inferior al mínimo establecido en esta ley, debiendo dejar constancia en la sentencia de las razones que motivaron su decisión.
Párrafo 2º
De las rebajas y aumentos de penas
ARTICULO 4°.- Para los efectos de rebajar, según corresponda, en uno o dos grados la pena asignada a los delitos contemplados en los artículos primero y tercero, el juez considerará, especialmente, alguna de las circunstancias siguientes:
a)La inmediata y espontánea confesión de participación en el delito prestada al Ministerio Público, aun cuando tal confesión no importe cooperación eficaz.
b)La escasa potencialidad dañina con que obró el imputado.
c)Si el acusado actuó solo en la perpetración del hecho delictuoso.
La rebaja de penalidad determinada en virtud de alguna de estas circunstancias, deberá ser fundada.
ARTICULO 5°.- No obstante lo dispuesto en el artículo anterior, no procederá rebaja de pena, o está podrá ser aumentada en un grado, atendida la entidad del hecho, si concurre alguna de las circunstancias siguientes:
a)Si el imputado formó parte de una agrupación o reunión de delincuentes que, sin incurrir en el delito de organización del artículo 22, agregó una mayor criminalidad al comportamiento de los malhechores.
b)Si el delincuente ha sido condenado por crimen o simple delito de carácter internacional, cualesquiera sea su naturaleza.
c)Si con ocasión de la comisión del delito contemplado en esta ley, incurrió, además, en otros delitos.
d)Si se utilizó violencia o armas en su comisión.
e)Si hubo aprovechamiento de personas exentas de responsabilidad penal o de menores inimputables en su perpetración.
f)Si el delito se cometió en las inmediaciones o en el interior de un establecimiento de enseñanza o en sitios a los que escolares y estudiantes acuden a realizar actividades educativas, deportivas o sociales, encontrándose éstos, en dichos lugares.
g)Si se perpetró en una institución deportiva, cultural o social, mientras éstas cumplen sus fines propios; o en sitios donde se estén realizando espectáculos públicos, actividades recreativas, culturales o sociales.
h)Si fue cometido en un centro hospitalario, asistencial, lugar de detención, recinto militar o policial.
i)Si se suministró, promovió o facilitó el uso o consumo de drogas o sustancias estupefacientes o psicotrópicas, a menores de dieciocho años de edad o a personas con sus facultades mentales disminuidas o perturbadas.
j)Si el hechor indujo, promovió o facilitó el uso o consumo de estupefacientes, psicotrópicos o hidrocarburos aromáticos, a personas a cuyo cargo o cuidado se encontraban.
k)Si el delito se cometió por funcionarios públicos aprovechando o abusando de su calidad de tal.
Lo dispuesto en el inciso primero de este artículo es sin perjuicio de la rebaja de pena que autorizan las modificatorias comunes de responsabilidad penal y la cooperación eficaz.
ARTICULO 5º.En los delitos contemplados en esta ley no procederá la atenuante de responsabilidad penal contenida en el número 7 del artículo 11 del Código Penal.
ARTICULO 6º.Para determinar si existe reincidencia respecto de los delitos castigados en esta ley, se considerarán las sentencias firmes dictadas en un Estado extranjero, aun cuando la pena impuesta no haya sido cumplida.
Párrafo 3º
De los delitos específicos
ARTICULO 7°.- El que suministre a menores de dieciocho años de edad, a cualquier título, productos que contengan hidrocarburos aromáticos, tales como benceno, tolueno u otras sustancias similares, en proporción suficiente para producir efectos tóxicos o psicotrópicos , a sabiendas de que están destinados a ser consumidos por dichas personas, incurrirá en la pena de presidio menor en su grado medio a máximo y multa de cuarenta o doscientas unidades tributarias mensuales.
Si sólo hubo imprudencia o negligencia culpable en el suministro, la sanción será reclusión o relegación menores en su grado mínimo o multa de veinte o cien unidades tributarias mensuales.
Atendidas las circunstancias del delito, podrá imponerse, además, la clausura a que hace referencia el artículo octavo de esta ley.
ARTICULO 8°.- El médico cirujano, odontólogo, médico veterinario o matrona, que a sabiendas recete alguna de las sustancias señaladas en el artículo primero, sin necesidad médica o terapéutica, será penado con presidio mayor en sus grados mínimo a medio y multa de cuarenta a cuatrocientas unidades tributarias mensuales.
Si obrare con imprudencia o negligencia culpable, la sanción será reclusión o relegación menores en su grado mínimo a medio y multa de veinte a cien unidades tributarias mensuales.
Procederá en estos casos la rebaja de pena contemplada en inciso 2° del artículo 1°, atendida la naturaleza de la sustancia que se prescriba, si la forma y circunstancias de la perpetración del hecho, así lo ameritan.
ARTICULO 9°.- El que encontrándose autorizado para suministrar a cualquier título las sustancias o drogas a que se refiere el artículo 1°; o las materias que sirvan para obtenerlas, lo hiciere en contravención a las disposiciones legales o reglamentarias que lo regulan, será sancionado con presidio mayor en sus grados mínimo a medio y multa de cuarenta a cuatrocientas unidades tributarias mensuales. Atendidas las circunstancias del delito, podrá imponerse, además, la medida de clausura temporal del establecimiento por un plazo no inferior a sesenta días ni superior a ciento veinte días, aún cuando el autor del hecho sea empleado o dependiente de cualquier modo en dicho establecimiento. En caso de reiteración, podrá imponerse la clausura definitiva de la entidad y la prohibición perpetua al autor de tales ilícitos, de participar en otro establecimiento de igual naturaleza.
ARTICULO 10°.- El que siembre, plante, cultive, o coseche especies vegetales del género cannabis u otras productoras de sustancias estupefacientes o psicotrópicas, careciendo de la debida autorización, incurrirá en la pena de presidio menor en su grado máximo a presidio mayor en su grado mínimo y multa de cuarenta a cuatrocientas unidades tributarias mensuales, a menos que se acredite que estuvieron destinadas al uso o consumo personal exclusivo y próximo en el tiempo, en cuyo caso sólo se aplicarán las normas de los artículos 48 y siguientes de esta ley.
Según la gravedad del hecho y las circunstancias personales del imputado, la pena podrá rebajarse en un grado.
ARTICULO 11.- La autorización a que se refiere el artículo anterior será otorgada por el Servicio Agrícola y Ganadero; y no podrá dársela a las personas acusadas o condenadas por alguno de los delitos contemplados en esta ley; o tratándose de personas jurídicas, cuando cualesquiera de sus socios, asociados o administradores, se encuentren en alguna de dichas situaciones.
Se suspenderá la autorización concedida por el solo ministerio de la ley, si con posterioridad a ésta se presenta la acusación; y se entenderá cancelada definitivamente, de igual modo, desde que se enc...</pre>


✅ Conexión a la base de datos cerrada.


In [1]:
import sqlite3
import pandas as pd
from IPython.display import display, Markdown

# --- CONFIGURACIÓN ---
# Asegúrate de que esta ruta sea correcta para tu archivo de base de datos.
DB_PATH = 'data/database/parlamento.db' 
pd.set_option('display.max_rows', 100)

# --- 1. CONEXIÓN A LA BASE DE DATOS ---
conn = None # Inicializamos la variable de conexión
try:
    # Se establece la conexión a la base de datos SQLite
    conn = sqlite3.connect(DB_PATH)
    print(f"✅ Conexión a la base de datos '{DB_PATH}' exitosa.")

    # --- 2. OBTENER Y MOSTRAR LAS TABLAS ---
    
    # Creamos un cursor para ejecutar consultas SQL
    cursor = conn.cursor()
    
    # Consultamos la tabla maestra para obtener los nombres de todas las tablas de usuario
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = cursor.fetchall() # Obtenemos todos los resultados

    print(f"\nSe encontraron {len(tablas)} tablas. Mostrando las 5 primeras filas de cada una:\n")

    # Iteramos sobre la lista de nombres de tablas encontradas
    for tabla in tablas:
        nombre_tabla = tabla[0] # El nombre de la tabla está en la primera posición de la tupla
        
        # Usamos display y Markdown para mostrar el nombre de la tabla como un título
        display(Markdown(f'### Tabla: `{nombre_tabla}`'))
        
        # Creamos la consulta SQL para seleccionar las 5 primeras filas
        query = f"SELECT * FROM {nombre_tabla} LIMIT 5"
        
        # Usamos pandas para ejecutar la consulta y leer el resultado en un DataFrame
        df = pd.read_sql_query(query, conn)
        
        # Mostramos el DataFrame
        display(df)
        print("\n" + "="*50 + "\n") # Añadimos un separador para mayor claridad

except sqlite3.Error as e:
    # Si ocurre algún error durante la conexión o consulta, se imprime el error
    print(f"❌ Error de base de datos: {e}")

finally:
    # --- 3. CERRAR LA CONEXIÓN ---
    if conn:
        conn.close()
        print("✅ Conexión a la base de datos cerrada.")

✅ Conexión a la base de datos 'data/database/parlamento.db' exitosa.

Se encontraron 27 tablas. Mostrando las 5 primeras filas de cada una:



### Tabla: `dim_parlamentario`

,mp_uid,nombre_completo,nombre_propio,apellido_paterno,apellido_materno,genero,fecha_nacimiento,lugar_nacimiento,diputadoid,senadorid,bcn_person_id,bcn_uri,url_foto,twitter_handle,sitio_web_personal,profesion,url_historia_politica,fecha_extraccion
0,1,Carlos Bombal Otaegui,Carlos,Bombal,Otaegui,Masculino,None,None,415,None,2721,http://datos.bcn.cl/recurso/persona/2721,https://www.bcn.cl/laborparlamentaria/imagen/2...,None,None,Abogado,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
1,2,Ramón Arias,Ramón,Arias,None,Masculino,None,None,None,None,2483,http://datos.bcn.cl/recurso/persona/2483,https://www.bcn.cl/laborparlamentaria/imagen/2...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
2,3,José Plácido Musalem Saffie,José Plácido,Musalem,Saffie,Masculino,None,None,None,None,2315,http://datos.bcn.cl/recurso/persona/2315,https://www.bcn.cl/laborparlamentaria/imagen/2...,None,None,Abogado,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
3,4,Arístides Benavente Carvajal,Arístides,Benavente,Carvajal,Masculino,None,None,None,None,458,http://datos.bcn.cl/recurso/persona/458,https://www.bcn.cl/laborparlamentaria/imagen/4...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06
4,5,Juan de Dios Peñafiel Varela,Juan de Dios,Peñafiel,Varela,Masculino,None,None,None,None,597,http://datos.bcn.cl/recurso/persona/597,https://www.bcn.cl/laborparlamentaria/imagen/5...,None,None,None,https://www.bcn.cl/historiapolitica/resenas_pa...,2025-09-06


### Tabla: `sqlite_sequence`

,name,seq
0,dim_ministerios,29
1,dim_parlamentario,3964
2,entity_sources,17
3,bill_tramites,232
4,parlamentario_mandatos,10061


...

### Tabla: `speech_turns`

,speech_id,mp_uid,sesion_votacion_id,comision_id,texto,fecha,tema,url_video,inicio_seg,fin_seg


### Tabla: `interactions`

,interaction_id,source_uid,target_uid,tipo,fecha,fuente,snippet




✅ Conexión a la base de datos cerrada.
